In [24]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [23]:
import plotly.graph_objects as go
import pandas as pd
from datetime import datetime
import yfinance as yf
import ipywidgets as widgets
import ipywidgets as ipw
from IPython.display import HTML, display,Javascript,clear_output
from ipywidgets import interact, interactive, fixed, interact_manual
# https://www.youtube.com/watch?v=abXuRM0QVD0&ab_channel=Elliott


def plot_chart(stock_df):

    import plotly.express as px
    fig = go.Figure()
    Candlestick=go.Candlestick(x=stock_df.index,
                    open=stock_df['Open'],
                    high=stock_df['High'],
                    low=stock_df['Low'],
                    close=stock_df['Close'],name='Candlestick')

    #fig = px.line(google_df, x=google_df.index, y="30moving average")

    ma150=go.Scatter(x=stock_df.index, y=stock_df["150 days moving average"],
                        line=dict(color='yellow', width=4),
                        name='150 moving average')
    
    ma50=go.Scatter(x=stock_df.index, y=stock_df["50 days moving average"],
                        line=dict(color='blue', width=4),
                        name='50 moving average')



    data=[Candlestick,ma50,ma150]
    layout = go.Layout( 
        title=stock_df.name+" Stock Price",
        title_x=0.5,
        xaxis_title="Date",
        yaxis_title="Stock Price",
        legend_title="Legend",
        font=dict(
            family="Courier New, monospace",
            size=12,
            color="RebeccaPurple"
        ),
        
        legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    ),
        yaxis=dict(
           autorange = True,
           fixedrange= False
       ),
        width=1020,    height=700,)

    figure = go.Figure(data=data, layout=layout)
    f2 = go.FigureWidget(figure)
    display(f2)
    
def linechart(stock_df):
    fig = go.Figure([go.Scatter(x=stock_df.index, y=stock_df['High'])])
    fig.update_layout(
        title=stock_df.name+" Stock Price",
        title_x=0.5,
        xaxis_title="Date",
        yaxis_title="Stock Price",
        legend_title="Legend",
        font=dict(
            family="Courier New, monospace",
            size=12,
            color="RebeccaPurple"
        )
    )
    fig.show()


def findstable(df,date,last,percentage=0.05):
    df=df.loc[date:]
    start=date
    price=df.loc[date,'Open']
    interval=[price*(1-percentage),price*(1+percentage)]
    list1=[]
    day_count=0
    
    for date,row in df.iterrows():
        if (interval[0]<=row['High']<=interval[1]) and (interval[0]<=row['Low']<=interval[1]) and date!=df.index[-1]:
            day_count+=1
        else:
            if day_count>=last or date==df.index[-1]:
                list1.append((price,start,date,interval,day_count))
            start=date    
            price=row['Open']
            interval=[price*(1-percentage),price*(1+percentage)]
            day_count=0
    return pd.DataFrame(list1,columns=['price','Start_date','End_date',str(percentage*100)+'% Interval','day count'])


        
def selectStock(df,Stock_Search,date):
    date=date.value.strftime("%Y-%m-%d")
    temp=df.iloc[:, df.columns.get_level_values(1)==Stock_Search.value].copy()
    temp=temp.loc[date:]
    temp=temp.stack().droplevel(1)
    temp.name=Stock_Search.label
    plot_chart(temp)
    #linechart(stock_df)    
    df=findstable(temp,date,15)
    del temp
    display(df)
    


stock_df=yf.download(  # or pdr.get_data_yahoo(...
        # tickers list or string as well
        tickers = "goog msft mpngf",

        # use "period" instead of start/end
        # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        # (optional, default is '1mo')
        period = "max")

stock_list=stock_df.columns.get_level_values(1).unique()
stock_df[[('50 days moving average',stock) for stock in stock_list]]=stock_df.xs('Close', axis=1, level=0).rolling(window=50).mean()
stock_df[[('150 days moving average',stock) for stock in stock_list]]=stock_df.xs('Close', axis=1, level=0).rolling(window=150).mean()





date=ipw.DatePicker(
    description='Start Date',
    disabled=False,
    value=datetime(year=2020,month=1,day=3)
)

Dropdown=ipw.Dropdown( 
    options=[('Google','GOOG'),('Microsoft Corporation','MSFT'),('Meituan','MPNGF')],description='Stock:') #=[('One', 1), ('Two', 2), ('Three', 3)],

btn=ipw.Button(description='Search',
           disabled=False,
           buttion_style='',
           tooltip='Click me'
)

out = widgets.Output()

search_row=ipw.HBox([Dropdown,date,btn])

@out.capture()
def is_onclick(b):
    clear_output(True)
    display(search_row)
    selectStock(stock_df,Dropdown,date)
    
btn.on_click(is_onclick)
btn.click()
out

[*********************100%***********************]  3 of 3 completed


Output()